In [1]:
import nuclio 

In [7]:
# nuclio: start-code

In [2]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import os
import datetime

In [4]:
def handler(context, event):
    context.logger.info("Reading features from feature vector")
    # Reading the data from feature service
    start = datetime.datetime.now()-datetime.timedelta(5)
    end = datetime.datetime.now()
    df = fs.get_offline_features("stocks-vec",start_time = start, end_time = end,entity_timestamp_column="Datetime").to_dataframe()
    df = df[:20 if df.shape[0]>20 else df.shape[0]] 
#     df["Datetime"] = df["Datetime"].apply(lambda x:datetime.datetime.strftime(x,"%Y-%m-%d %H:%M:%S"))
    df = df.drop(["Content","Link"],axis = 1)
    data = df.fillna(0).values.tolist()
    
    columns = [{'text': key, 'type': 'object'} for key in df.columns]
    
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response

In [5]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-' + os.getenv('V3IO_USERNAME')))
    setattr(context, 'ARTIFACT_PATH', os.getenv('ARTIFACT_PATH', './'))
    mlrun.set_environment(project=context.PROJECT_NAME, artifact_path = context.ARTIFACT_PATH)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc', 'INTC' : 'intel-corp'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.*",
                "news.*"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.spec.with_indexes=True
    vector.save()

In [6]:
# nuclio: end-code

In [ ]:
mlrun.set_environment(project="stocks-" + os.getenv('V3IO_USERNAME'))

In [ ]:
# test local
init_context(context)
event = ""
df = handler(context,event)

## test remote deployment

In [1]:
import mlrun
import os
fn = mlrun.code_to_function('vector_reader',
                      handler='handler',kind='nuclio',image='mlrun/mlrun:0.6.5')

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : "stocks-" + os.getenv('V3IO_USERNAME'),
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream',
             'ARTIFACT_PATH' :  "v3io:///projects/" + "stocks-" + os.getenv('V3IO_USERNAME') + '/artifacts'})
fn.spec.max_replicas = 2

fn.apply(mlrun.platforms.v3io_cred())
fn.apply(mlrun.mount_v3io())

In [2]:
addr = fn.deploy(project="stocks-" + os.getenv('V3IO_USERNAME'))

> 2021-09-29 07:06:23,740 [info] Starting remote function deploy
2021-09-29 07:06:24  (info) Deploying function
2021-09-29 07:06:24  (info) Building
2021-09-29 07:06:24  (info) Staging files and preparing base images
2021-09-29 07:06:24  (info) Building processor image
2021-09-29 07:06:28  (info) Build complete
2021-09-29 07:06:50  (info) Function deploy complete
> 2021-09-29 07:06:51,516 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-stocks-dani-vector-reader.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.dev8.lab.iguazeng.com:31963']}


In [ ]:
!curl {addr}